In [1]:
import xarray as xr
import netCDF4 as nc

In [2]:
import dask.dataframe as dd

In [3]:
from dask.distributed import Client, LocalCluster

In [4]:
import matplotlib.pyplot as plt

In [5]:
import numpy as np

In [6]:
import os
import time

In [7]:
from testcases import testcases

In [8]:
USERNAME = "x-twhite77"
NUM_REPETITIONS = 5
NUM_WORKERS = 64
MEMORY = "1000GB"

In [9]:
client = None

In [10]:
def initialize() ->None:
    cluster = LocalCluster(n_workers=NUM_WORKERS, memory_limit=1/(NUM_WORKERS + 1))
    client = Client(cluster)

def shutdown() ->None:
    client.close()

In [11]:
ds = xr.open_mfdataset("/anvil/datasets/ncar/AORC_Forcing/2016/201601*", engine = "netcdf4")

In [12]:
ds

<xarray.Dataset>
Dimensions:         (time: 744, reference_time: 1, x: 4608, y: 3840)
Coordinates:
  * time            (time) datetime64[ns] 2016-01-01 ... 2016-01-31T23:00:00
  * reference_time  (reference_time) datetime64[ns] 2016-01-01
  * x               (x) float64 -2.303e+06 -2.302e+06 ... 2.303e+06 2.304e+06
  * y               (y) float64 -1.92e+06 -1.919e+06 ... 1.918e+06 1.919e+06
Data variables:
    crs             (time) |S1 b'' b'' b'' b'' b'' b'' ... b'' b'' b'' b'' b''
    U2D             (time, y, x) float64 dask.array<chunksize=(1, 768, 922), meta=np.ndarray>
    V2D             (time, y, x) float64 dask.array<chunksize=(1, 768, 922), meta=np.ndarray>
    LWDOWN          (time, y, x) float64 dask.array<chunksize=(1, 768, 922), meta=np.ndarray>
    RAINRATE        (time, y, x) float32 dask.array<chunksize=(1, 768, 922), meta=np.ndarray>
    T2D             (time, y, x) float64 dask.array<chunksize=(1, 768, 922), meta=np.ndarray>
    Q2D             (time, y, x) float64 dask.array<chunksize=(1, 768, 922), meta=np.ndarray>
    PSFC            (time, y, x) float64 dask.array<chunksize=(1, 768, 922), meta=np.ndarray>
    SWDOWN          (time, y, x) float64 dask.array<chunksize=(1, 768, 922), meta=np.ndarray>
    LQFRAC          (time, y, x) float64 dask.array<chunksize=(1, 768, 922), meta=np.ndarray>
Attributes:
    model_output_valid_time:    2016-01-01_00:00:00
    model_initialization_time:  2016-01-01_00:00:00
    NWM_version_number:         v2.1
    model_configuration:        AORC
    model_output_type:          forcing
    model_total_valid_times:    744.0

In [13]:
dataset = ds.chunk({"time":24*7, "x":800,"y":1000})

In [14]:
dataset

<xarray.Dataset>
Dimensions:         (time: 744, reference_time: 1, x: 4608, y: 3840)
Coordinates:
  * time            (time) datetime64[ns] 2016-01-01 ... 2016-01-31T23:00:00
  * reference_time  (reference_time) datetime64[ns] 2016-01-01
  * x               (x) float64 -2.303e+06 -2.302e+06 ... 2.303e+06 2.304e+06
  * y               (y) float64 -1.92e+06 -1.919e+06 ... 1.918e+06 1.919e+06
Data variables:
    crs             (time) |S1 dask.array<chunksize=(168,), meta=np.ndarray>
    U2D             (time, y, x) float64 dask.array<chunksize=(168, 1000, 800), meta=np.ndarray>
    V2D             (time, y, x) float64 dask.array<chunksize=(168, 1000, 800), meta=np.ndarray>
    LWDOWN          (time, y, x) float64 dask.array<chunksize=(168, 1000, 800), meta=np.ndarray>
    RAINRATE        (time, y, x) float32 dask.array<chunksize=(168, 1000, 800), meta=np.ndarray>
    T2D             (time, y, x) float64 dask.array<chunksize=(168, 1000, 800), meta=np.ndarray>
    Q2D             (time, y, x) float64 dask.array<chunksize=(168, 1000, 800), meta=np.ndarray>
    PSFC            (time, y, x) float64 dask.array<chunksize=(168, 1000, 800), meta=np.ndarray>
    SWDOWN          (time, y, x) float64 dask.array<chunksize=(168, 1000, 800), meta=np.ndarray>
    LQFRAC          (time, y, x) float64 dask.array<chunksize=(168, 1000, 800), meta=np.ndarray>
Attributes:
    model_output_valid_time:    2016-01-01_00:00:00
    model_initialization_time:  2016-01-01_00:00:00
    NWM_version_number:         v2.1
    model_configuration:        AORC
    model_output_type:          forcing
    model_total_valid_times:    744.0

In [15]:
dataset['RAINRATE']

<xarray.DataArray 'RAINRATE' (time: 744, y: 3840, x: 4608)>
dask.array<rechunk-merge, shape=(744, 3840, 4608), dtype=float32, chunksize=(168, 1000, 800), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 2016-01-01 ... 2016-01-31T23:00:00
  * x        (x) float64 -2.303e+06 -2.302e+06 ... 2.303e+06 2.304e+06
  * y        (y) float64 -1.92e+06 -1.919e+06 -1.918e+06 ... 1.918e+06 1.919e+06
Attributes:
    cell_methods:    time: mean
    remap:           remapped via ESMF regrid_with_weights: Bilinear
    grid_mapping:    crs
    esri_pe_string:  PROJCS["Lambert_Conformal_Conic",GEOGCS["GCS_Sphere",DAT...
    proj4:           +proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1=30.0...
    units:           mm s^-1
    standard_name:   precipitation_flux
    long_name:       Surface Precipitation Rate

In [16]:
# dask client
client = None

# TODO
# Measure time to create and save chunks
# Test across the time dimension 

def initialize() ->None:
  cluster = LocalCluster(n_workers=NUM_WORKERS, memory_limit=1/(NUM_WORKERS + 1))
  client = Client(cluster)

def shutdown() ->None:
  client.close()


def GetTestTime(datatset, testcases) -> int:
  times = []

  for func in testcases:
    print(f"\t\tTest case: {func['name']}")
    timetoadd = 0
    startTime = time.time()

    for i in range(NUM_REPETITIONS):
      func['func'](datatset)
      
    endTime = time.time()
    timetoadd += endTime - startTime
    timetoadd = timetoadd / NUM_REPETITIONS

    times.append(timetoadd)
    
  return times

def RunTestCases(pictureName, testcases, chunkSchemes) -> None:
  print("Running benchmark test cases")
  times = []

  for scheme in chunkSchemes:
    fullpath = os.path.join(f"/anvil/scratch/{USERNAME}/", f"{scheme['name']}.zarr")
    print(f"\tTesting scheme: {scheme['name']}")
    ds = xr.open_zarr(fullpath)
    times.append(GetTestTime(ds, testcases))

  times = np.array(times)

  for idx, test in enumerate(testcases):
    plt.figure(figsize=(12, 6))

    # Create a line plot
    plt.plot([scheme['name'] for scheme in chunkSchemes], times[:, idx])

    # Add labels and title
    plt.xlabel('Chunk Size')
    plt.ylabel(f'Time for test case')
    plt.title(f'{testcases[idx]}')

    # Save the chart to a file (e.g., as a PNG image)
    plt.savefig(f'{testcases[idx]}_{pictureName}.png')

    # avoid overlap
    plt.close()

def Rechunk(chunkSchemes) -> None:
  print("Attempting to re-chunk dataset")

  rrt = None
  for scheme in chunkSchemes:
    if not os.path.exists(f"/anvil/scratch/{USERNAME}/{scheme['name']}.zarr"):

      if rrt is None:
        print("Missing scheme found! Loading entire dataset...")
        ds = xr.open_mfdataset("/anvil/projects/x-cis220065/x-cybergis/compute/AORC_Forcing/2016/*.LDASIN_DOMAIN1", engine = "netcdf4", combine= "nested", concat_dim="Time", parallel = "True")
        rrt = ds["RAINRATE"]

      print(f"Re-chunking using scheme {scheme['name']}")

      rrt = rrt.chunk(scheme['scheme'])
      rrt.to_zarr(f"/anvil/scratch/{USERNAME}/{scheme['name']}.zarr", mode="w")
  
if __name__ == "__main__" :
  #Start the client
  initialize()

  # Test chunk schemes across time
  chunkSchemes = [
    {'name': "10x100x100", 'scheme': {"Time": 10, "south_north": 100, "west_east": 100}},
    {'name': "50x300x300", 'scheme': {"Time": 50, "south_north": 300, "west_east": 300}},
    {'name': "100x600x600", 'scheme': {"Time": 100, "south_north": 600, "west_east": 600}},
    {'name': "150x900x900", 'scheme': {"Time": 150, "south_north": 900, "west_east": 900}},
    {'name': "200x1200x1200", 'scheme': {"Time": 200, "south_north": 1200, "west_east": 1200}},
    ]

  Rechunk(chunkSchemes)
  RunTestCases("chunk_test", testcases, chunkSchemes)

  shutdown()

Attempting to re-chunk dataset
Running benchmark test cases
	Testing scheme: 10x100x100
		Test case: Max_Rain_Test


/home/x-twhite77/.conda/envs/rocky8.7/conda-4.12.0/labpacks/lib/python3.9/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 12.86 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/home/x-twhite77/.conda/envs/rocky8.7/conda-4.12.0/labpacks/lib/python3.9/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 12.86 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/home/x-twhite77/.conda/envs/rocky8.7/conda-4.12.0/labpacks/lib/python3.9/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 12.86 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/home/x-twhite77/.conda/envs/rocky8.7/conda-4.12.0/labpacks/lib/python3.9/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 12.86 MiB.
This may caus

		Test case: Average_200_sqr
		Test case: Average_400_sqr
		Test case: Average_800_sqr
		Test case: Average_1600_sqr


2024-04-22 12:37:05,663 - distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
2024-04-22 12:37:24,473 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2024-04-22 12:37:38,173 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2024-04-22 12:37:42,502 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2024-04-22 12:37:56,726 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2024-04-22 12:38:14,458 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2024-04-22 12:38:18,950 - distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
2024-04-22 12:38:30,661 - distributed.utils_perf - WARNING - full garbage collections took

	Testing scheme: 50x300x300
		Test case: Max_Rain_Test


...

		Test case: Average_200_sqr


2024-04-22 12:39:50,852 - distributed.scheduler - ERROR - Couldn't gather keys: {('mean_agg-aggregate-e6b1f435515df61b99d169ee0741f4dd', 0, 0): 'forgotten'}
2024-04-22 12:39:50,854 - distributed.client - WARNING - Couldn't gather 1 keys, rescheduling (('mean_agg-aggregate-e6b1f435515df61b99d169ee0741f4dd', 0, 0),)


		Test case: Average_400_sqr
		Test case: Average_800_sqr
		Test case: Average_1600_sqr


2024-04-22 12:39:53,381 - distributed.worker.memory - WARNING - Worker is at 88% memory usage. Pausing worker.  Process memory: 3.25 GiB -- Worker memory limit: 3.69 GiB
2024-04-22 12:39:53,437 - distributed.worker.memory - WARNING - Worker is at 36% memory usage. Resuming worker. Process memory: 1.34 GiB -- Worker memory limit: 3.69 GiB
2024-04-22 12:39:53,899 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 2.62 GiB -- Worker memory limit: 3.69 GiB
2024-04-22 12:39:54,210 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memo

AttributeError: 'NoneType' object has no attribute 'close'

		Test case: Max_Rain_Test
		Test case: Average_200_sqr
		Test case: Average_400_sqr
		Test case: Average_800_sqr
		Test case: Average_1600_sqr


In [20]:
def GetTestTime(datatset, testcases) -> int:
    times = []

    for func in testcases:
        print(f"\t\tTest case: {func['name']}")
        timetoadd = 0
        startTime = time.time()

        for i in range(NUM_REPETITIONS):
         func['func'](datatset)
      
        endTime = time.time()
        timetoadd += endTime - startTime
        timetoadd = timetoadd / NUM_REPETITIONS

        times.append(timetoadd)
    
    return times


In [27]:
def RunTestCases(pictureName, testcases, chunkSchemes) -> None:
    print("Running benchmark test cases")
    times = []

for scheme in chunkSchemes:
    fullpath = os.path.join(f"/anvil/scratch/{USERNAME}/", f"{scheme['name']}.zarr")
    print(f"\tTesting scheme: {scheme['name']}")
    ds = xr.open_zarr(fullpath)
    times.append(GetTestTime(ds, testcases))

times = np.array(times)

NameError: name 'chunkSchemes' is not defined

In [31]:
 for idx, test in enumerate(testcases):
    plt.figure(figsize=(12, 6))

    # Create a line plot
    plt.plot([scheme['name'] for scheme in chunkSchemes], times[:, idx])

    # Add labels and title
    plt.xlabel('Chunk Size')
    plt.ylabel(f'Time for test case')
    plt.title(f'{testcases[idx]['name']}')

    # Save the chart to a file (e.g., as a PNG image)
    plt.savefig(f'{testcases[idx]['name']}_{pictureName}.png')

    # avoid overlap
    plt.close()

def Rechunk(chunkSchemes) -> None:
    print("Attempting to re-chunk dataset")

SyntaxError: f-string: unmatched '[' (3401181214.py, line 10)

In [33]:
def Rechunk(chunkSchemes) -> None:
print("Attempting to re-chunk dataset")

  rrt = None
    for scheme in chunkSchemes:
    if not os.path.exists(f"/anvil/scratch/{USERNAME}/{scheme['name']}.zarr"):

    if rrt is None:
        print("Missing scheme found! Loading entire dataset...")
        ds = xr.open_mfdataset("/anvil/projects/x-cis220065/x-cybergis/compute/AORC_Forcing/2016/*.LDASIN_DOMAIN1", engine = "netcdf4", combine= "nested", concat_dim="Time", parallel = "True")
        rrt = ds["RAINRATE"]

print(f"Re-chunking using scheme {scheme['name']}")

      rrt = rrt.chunk(scheme['scheme'])
        rrt.to_zarr(f"/anvil/scratch/{USERNAME}/{scheme['name']}.zarr", mode="w")

IndentationError: expected an indented block (1250284484.py, line 2)

In [34]:
if __name__ == "__main__" :
  #Start the client
  initialize()

  # Test chunk schemes across time
  chunkSchemes = [
    {'name': "10x100x100", 'scheme': {"Time": 10, "south_north": 100, "west_east": 100}},
    {'name': "50x300x300", 'scheme': {"Time": 50, "south_north": 300, "west_east": 300}},
    {'name': "100x600x600", 'scheme': {"Time": 100, "south_north": 600, "west_east": 600}},
    {'name': "150x900x900", 'scheme': {"Time": 150, "south_north": 900, "west_east": 900}},
    {'name': "200x1200x1200", 'scheme': {"Time": 200, "south_north": 1200, "west_east": 1200}},
    ]

Rechunk(chunkSchemes)
RunTestCases("chunk_test", testcases, chunkSchemes)

shutdown()

NameError: name 'Rechunk' is not defined

In [ ]:
plt.plot(dataset.groupby(dataset['time'].dt.year)['RAINRATE'].mean())